In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from quantum_lib import (
    QuantumInformationField, 
    ZeemanSpectroscopicClassifier
)

# 1. Load Data
data = load_iris()
X_raw = data.data  # Oryginalne dane
y = data.target
target_names = data.target_names

# 2. Preprocessing: Scaler (Mapowanie do zakresu [0, pi])
# To krytyczny krok dla bramek obrotowych RY/RZ
scaler = MinMaxScaler(feature_range=(0, np.pi))
X = scaler.fit_transform(X_raw)

# Just Ry embedding analysis

In [2]:
n_features = X.shape[1]
print(f"analysis for: {n_features} features")
# for benchmark
qf = QuantumInformationField(n_features=n_features)

results_b, metrics_b, y_pred_b, y_scores_b= qf.analyze(X, y, None, target_names)
qf.print_quantum_stats(results_b)
qf.print_mismatch_report(y, y_pred_b, y_scores_b)

analysis for: 4 features

KWANTOWA ANALIZA SPEKTRALNA POLA (BOZONY)
     Class purity    gap hbar_eff entropy lambda_0
    setosa 0.8472 2.6368   0.1528  0.3447   0.9181
versicolor 0.7968 2.4795   0.2032  0.4615   0.8892
 virginica 0.7345 2.2916   0.2655  0.6019   0.8519

--- RAPORT BŁĘDÓW (BOZONOWY) ---
 ID       True       Pred Fid Gap Fid setosa Fid versicolor Fid virginica Diagnosis
 50 versicolor  virginica  0.0133     0.0913         0.7370        0.7502 Ambiguity
 52 versicolor  virginica  0.0598     0.0755         0.7735        0.8333   Anomaly
 70 versicolor  virginica  0.0176     0.0922         0.8255        0.8431 Ambiguity
 77 versicolor  virginica  0.1289     0.0586         0.8011        0.9301   Anomaly
106  virginica versicolor  0.2692     0.1076         0.7301        0.4609   Anomaly
119  virginica versicolor  0.1834     0.0621         0.8172        0.6338   Anomaly
133  virginica versicolor  0.0663     0.0886         0.9182        0.8520   Anomaly
134  virginica versico

# J optimalization 

In [7]:
q_field = QuantumInformationField(n_features=X.shape[1])
#optimal_J = q_field.train_field(X, y, method="gap", restarts=3)
optimal_J = np.array([ 1.83729462,  0.00503822,  0.20411079, -0.72805969,  1.27776266, 4.23852519])
results, metrics, y_pred, y_scores = q_field.analyze(X, y, optimal_J, target_names)
q_field.print_quantum_stats(results)
q_field.print_mismatch_report(y, y_pred, y_scores)


KWANTOWA ANALIZA SPEKTRALNA POLA (BOZONY)
     Class purity    gap hbar_eff entropy lambda_0
    setosa 0.9712 4.7155   0.0288  0.0925   0.9854
versicolor 0.9376 3.8493   0.0624  0.1741   0.9681
 virginica 0.8965 3.5289   0.1035  0.2725   0.9463

--- RAPORT BŁĘDÓW (BOZONOWY) ---
 ID       True       Pred Fid Gap Fid setosa Fid versicolor Fid virginica Diagnosis
 70 versicolor  virginica  0.0292     0.4750         0.9269        0.9561 Ambiguity
 77 versicolor  virginica  0.0396     0.4687         0.9391        0.9787 Ambiguity
106  virginica versicolor  0.0678     0.4701         0.8942        0.8264   Anomaly
119  virginica versicolor  0.0847     0.5238         0.9457        0.8610   Anomaly
133  virginica versicolor  0.0386     0.5533         0.9829        0.9444 Ambiguity
134  virginica versicolor  0.0809     0.6037         0.9805        0.8996   Anomaly


In [8]:
metrics

{'accuracy': 0.96}

# ZEEMAN 

In [9]:
mismatches_idx = np.where(y != y_pred)[0]

if len(mismatches_idx) > 0:
    print(f"\nZEEMAN Diagnostics {len(mismatches_idx)} ...")
    
    # Inicjalizacja klasyfikatora spektroskopowego
    zeeman = ZeemanSpectroscopicClassifier(
        n_qubits=q_field.n_qubits, 
        centroids=q_field.centroids, 
        optimal_J=optimal_J
    )
    
    zeeman_report = []
    for idx in mismatches_idx:
        field_type, angle = zeeman.recover_sample(X[idx], y[idx])
        severity, description = zeeman.diagnose(field_type, angle)
        
        zeeman_report.append({
            'ID': idx,
            'True Class': target_names[y[idx]],
            'Prediction': target_names[y_pred[idx]],
            'Corrections': field_type,
            'Angel': f"{angle:.4f}" if angle else "N/A",
            'Diagnosis': severity
        })
    
    print("\n--- ZEEMAN SPECTRAL RAPORT ---")
    print(pd.DataFrame(zeeman_report).to_string(index=False))
else:
    print("\nNo Anomaly")


ZEEMAN Diagnostics 6 ...

--- ZEEMAN SPECTRAL RAPORT ---
 ID True Class Prediction Corrections   Angel Diagnosis
 70 versicolor  virginica           Y -0.7250   ANOMALY
 77 versicolor  virginica           Y -0.8861   ANOMALY
106  virginica versicolor           Z  0.0806     NOISE
119  virginica versicolor           Z  0.0806     NOISE
133  virginica versicolor           Z  0.0806     NOISE
134  virginica versicolor           Z  0.0806     NOISE
